In [1]:
import dask.dataframe as ddf
import collections
import pandas as pd
import json
import os
import numpy as np
import re
import hashlib

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
parquetdir = '../../tcm-columns-add-main/'

In [3]:
usernames = ddf.read_parquet(os.path.join(parquetdir,'contributor.username')).compute()
page_title = ddf.read_parquet(os.path.join(parquetdir,'page.title')).compute()

In [4]:
comment = ddf.read_parquet(os.path.join(parquetdir,'revision.comment')).compute()
revision_text = ddf.read_parquet(os.path.join(parquetdir,'revision.text')).compute()
contributor = usernames.drop(columns={'dir0'})

In [5]:
# filter out bots
def is_bot(text):
    text = text.split(' ')
    for item in text:
        if item.lower().endswith('bot'):
            return True
    return False

In [6]:
contributor['is_bot'] = contributor['contributor.username'].apply(is_bot)

In [7]:
contributor = contributor[~contributor['is_bot']]

In [8]:
page_user_info = contributor.join(page_title).drop(columns={'dir0'})

In [9]:
# # get total edits
# total_edits = page_user_info.groupby(['contributor.username','page.title']).count().\
# rename(columns={'is_bot':'total.contributions'})

In [11]:
# # former info
# former = ddf.read_parquet('../../intermediate-result/TCM/TCM-former-info').compute()

In [12]:
# sig info
amr_info = ddf.read_parquet('../../intermediate-result/TCM/TCM-added-modified-removed').compute()

In [13]:
added_text = amr_info[['new.text.added']]

In [14]:
def get_len(l):
    l = ' '.join(l)
    return len(l.split(' '))

In [15]:
added_text['len'] = added_text['new.text.added'].apply(get_len)

<ipython-input-15-af370a556aa9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  added_text['len'] = added_text['new.text.added'].apply(get_len)


In [16]:
added_text = added_text[added_text['len']>=20]

In [17]:
sig_df = added_text.join(page_title).join(contributor)

In [18]:
sig_df = sig_df.groupby(['contributor.username','page.title']).agg({'len':'count'}).\
rename(columns={'len':'sig.contributions'})

In [19]:
sig_res = sig_df.reset_index()

In [20]:
sig_res = ddf.from_pandas(sig_res,chunksize=10000)

In [21]:
sig_res.to_parquet('../../intermediate-result/TCM/sig-contrib-info')

In [22]:
editor_sig_contrib = sig_df.reset_index().groupby('contributor.username').agg({'sig.contributions':'sum'})

In [23]:
editor_sig_contrib = editor_sig_contrib.sort_values('sig.contributions',ascending=False)

In [24]:
# sig contribution stat - used to select editors based on sig contributions

In [25]:
test = [100,50,10,9,8,7,6,5,4,3,2,1,0]

In [26]:
print('There are {0} users in total'.format(len(page_user_info['contributor.username'].unique())))
for i in test:
    tmp = editor_sig_contrib[editor_sig_contrib['sig.contributions']>=i]
    print('There are {0} users with a total significant contribution greater than or equal to {1}'.format(len(tmp),i))

There are 22469 users in total
There are 82 users with a total significant contribution greater than or equal to 100
There are 158 users with a total significant contribution greater than or equal to 50
There are 678 users with a total significant contribution greater than or equal to 10
There are 751 users with a total significant contribution greater than or equal to 9
There are 829 users with a total significant contribution greater than or equal to 8
There are 945 users with a total significant contribution greater than or equal to 7
There are 1119 users with a total significant contribution greater than or equal to 6
There are 1328 users with a total significant contribution greater than or equal to 5
There are 1655 users with a total significant contribution greater than or equal to 4
There are 2240 users with a total significant contribution greater than or equal to 3
There are 3627 users with a total significant contribution greater than or equal to 2
There are 8896 users with 

In [27]:
# use 10 as the cutoff

In [28]:
editor_sig_contrib = editor_sig_contrib[editor_sig_contrib['sig.contributions']>=10]

In [29]:
editor_sig_contrib.to_parquet('../../intermediate-result/TCM/editors-with-sig-contrib-at-least-10')